<a href="https://colab.research.google.com/github/alexanderportilla96/Sy2025II/blob/main/Streamlit_parcial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# ==============================================================================
# PROYECTO FINAL: DASHBOARD SEÑALES Y SISTEMAS (FULL)
# Corrección: Inclusión de todas las gráficas (Polos, Bode, Impulso, Escalón, Rampa)
#             para Lazo Abierto y Lazo Cerrado en el Punto 4.
# ==============================================================================
import os
import subprocess
import time
import re
import sys

# 1. LIMPIEZA
print("🧹 Limpiando procesos...")
!pkill -9 streamlit
!pkill -9 cloudflared
if os.path.exists("tunnel.log"): os.remove("tunnel.log")

# 2. INSTALACIÓN
print("🛠️ Instalando librerías...")
!pip install streamlit -q
!pip install yt-dlp -q

# 3. AUDIO EXTREMODURO
print("🎵 Verificando audio...")
import yt_dlp
audio_file = "extremoduro.wav"
if not os.path.exists(audio_file):
    try:
        ydl_opts = {
            'format': 'bestaudio/best', 'outtmpl': audio_file,
            'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav'}],
            'quiet': True, 'nocheckcertificate': True,
            'extractor_args': {'youtube': {'player_client': ['android', 'ios']}}
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download(["https://www.youtube.com/watch?v=pM3JPhWApvY"])
        print("✅ Audio descargado.")
    except: print("⚠️ Audio no descargado (Subir manual).")

# 4. GENERAR APP.PY
print("📝 Escribiendo aplicación completa...")
codigo_app = """
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
import scipy.io.wavfile as wav
import os

st.set_page_config(page_title="Solución Parcial SyS", layout="wide", page_icon="🎛️")
st.markdown(\"\"\"<style>.stApp { background-color: #0E1117; color: white; } h1,h2 {color: #00BFFF;}</style>\"\"\", unsafe_allow_html=True)

# --- NAVEGACIÓN ---
st.sidebar.title("🎛️ Navegación")
opcion = st.sidebar.radio("Ir a:",
    ["1. Modulación AM (Punto 1)",
     "2. Modelo Matemático (Punto 2)",
     "3. Equivalencia RLC (Punto 3)",
     "4. Simulación Completa (Punto 4)"])

# ==========================================
# PUNTO 1: MODULACIÓN AM
# ==========================================
if opcion == "1. Modulación AM (Punto 1)":
    st.title("📡 Punto 1: Demodulación Síncrona")
    c1, c2 = st.columns([1, 1])
    with c1:
        st.markdown("**Diagrama:**")
        if os.path.exists("image_343fab.png"): st.image("image_343fab.png")
        else: st.info("(Sube 'image_343fab.png')")
    with c2:
        st.markdown("**Modelo:**")
        st.latex(r"y(t) = \\frac{A_1}{2}m(t) + \\frac{A_1}{2}m(t)\\cos(4\\pi f_c t)")

    col1, col2 = st.columns([1, 2])
    with col1:
        fc = st.number_input("Portadora (Hz)", 5000, 20000, 15000, step=1000)
        cutoff = st.number_input("Filtro LPF (Hz)", 2000, 12000, 8000, step=500)
        dur = st.slider("Duración (s)", 2.0, 10.0, 5.0)
    with col2:
        data, fs = None, 44100
        if os.path.exists("extremoduro.wav"):
            if st.button("🔄 Cargar Extremoduro"): fs, data = wav.read("extremoduro.wav")
        if data is None:
            uploaded = st.file_uploader("Sube audio .wav:", type=['wav'])
            if uploaded: fs, data = wav.read(uploaded)

    if data is not None:
        if len(data.shape) > 1: data = np.mean(data, axis=1)
        data = data/np.max(np.abs(data))
        N = int(dur*fs)
        if len(data)>N: data=data[:N]
        t = np.arange(len(data))/fs

        carrier = np.cos(2*np.pi*fc*t)
        mixer = (data * carrier) * carrier
        f = np.fft.fftfreq(len(data), 1/fs)
        mask = np.abs(f) < cutoff
        recup = np.real(np.fft.ifft(np.fft.fft(mixer) * mask)) * 2.0

        tab1, tab2, tab3 = st.tabs(["Original", "Mixer", "Recuperada"])
        with tab1: st.line_chart(data[0:1000]); st.audio(data, sample_rate=fs)
        with tab2: st.line_chart(mixer[0:1000]); st.audio(mixer, sample_rate=fs)
        with tab3: st.line_chart(recup[0:1000]); st.audio(recup, sample_rate=fs)

# ==========================================
# PUNTO 2: MODELO MATEMÁTICO
# ==========================================
elif opcion == "2. Modelo Matemático (Punto 2)":
    st.title("📐 Punto 2: Modelo Masa-Resorte")
    c1, c2 = st.columns(2)
    with c1:
        if os.path.exists("image_b54a5e.png"): st.image("image_b54a5e.png")
        else: st.info("(Sube 'image_b54a5e.png')")
    with c2:
        st.latex(r"G(s) = \\frac{1}{ms^2 + cs + k}")

# ==========================================
# PUNTO 3: EQUIVALENCIA RLC
# ==========================================
elif opcion == "3. Equivalencia RLC (Punto 3)":
    st.title("⚡ Punto 3: Equivalencia Eléctrica")
    c1, c2 = st.columns(2)
    with c1:
        if os.path.exists("image_830a9f.png"): st.image("image_830a9f.png")
        else: st.info("(Sube 'image_830a9f.png')")
    with c2:
        st.latex(r"L = \\frac{m}{kC}, \\quad R = \\frac{m}{cC}")

# ==========================================
# PUNTO 4: SIMULACIÓN COMPLETA
# ==========================================
elif opcion == "4. Simulación Completa (Punto 4)":
    st.title("⚙️ Punto 4: Simulación de Sistemas")
    st.markdown("Análisis comparativo: **Lazo Abierto (OL)** vs **Lazo Cerrado (CL)**")

    # --- ENTRADAS ---
    c1, c2, c3 = st.columns(3)
    with c1:
        st.subheader("Parámetros")
        m = st.number_input("Masa (kg)", 0.1, 10.0, 1.0)
        k = st.number_input("Resorte (N/m)", 0.1, 20.0, 4.0)
        c = st.slider("Amortiguador (Ns/m)", 0.0, 10.0, 1.0)
    with c2:
        st.subheader("Equivalente Eléctrico")
        C_ref = st.number_input("Capacitor (F)", 0.1, 1.0, 0.1)
        L = m/(k*C_ref)
        R = m/(c*C_ref) if c>0 else np.inf
        st.metric("Inductor L", f"{L:.2f} H")
        st.metric("Resistor R", f"{R:.2f} Ω")
    with c3:
        st.subheader("Diagnóstico")
        wn = np.sqrt(k/m)
        zeta = c / (2 * np.sqrt(k*m))
        st.metric("Frecuencia Natural", f"{wn:.2f} rad/s")
        st.metric("Zeta", f"{zeta:.3f}")

    # --- DEFINICIÓN DE SISTEMAS ---
    # Lazo Abierto: G(s) = 1 / (ms^2 + cs + k)
    sys_ol = signal.TransferFunction([1], [m, c, k])

    # Lazo Cerrado (Realimentación Unitaria): T(s) = G / (1+G)
    # G = N/D -> T = N / (D+N)
    # Denominador CL = [m, c, k+1]
    sys_cl = signal.TransferFunction([1], [m, c, k+1])

    t = np.linspace(0, 20, 1000)

    # --- PESTAÑAS CON TODAS LAS GRÁFICAS ---
    tabs = st.tabs(["1. Mapa Polos/Ceros", "2. Diagrama de Bode", "3. Respuesta Impulso", "4. Respuesta Escalón", "5. Respuesta Rampa"])

    # 1. POLOS Y CEROS
    with tabs[0]:
        st.subheader("Mapa de Polos y Ceros")
        fig, ax = plt.subplots(figsize=(6, 5))
        # Polos OL
        ax.scatter(np.real(sys_ol.poles), np.imag(sys_ol.poles), marker='x', s=100, color='blue', label='Polos Lazo Abierto')
        # Polos CL
        ax.scatter(np.real(sys_cl.poles), np.imag(sys_cl.poles), marker='o', s=100, color='red', facecolors='none', label='Polos Lazo Cerrado')
        ax.axvline(0, color='k', lw=1); ax.axhline(0, color='k', lw=1)
        ax.grid(True, which='both', linestyle='--', alpha=0.5)
        ax.legend(); ax.set_xlabel('Real'); ax.set_ylabel('Imaginario')
        st.pyplot(fig)
        st.info("Nota: Las 'X' azules son el sistema original. Los 'O' rojos son el sistema realimentado.")

    # 2. BODE
    with tabs[1]:
        st.subheader("Diagrama de Bode")
        w, mag_ol, phase_ol = signal.bode(sys_ol)
        w_cl, mag_cl, phase_cl = signal.bode(sys_cl)

        fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(8, 6))

        # Magnitud
        ax1.semilogx(w, mag_ol, 'b', label='Lazo Abierto')
        ax1.semilogx(w_cl, mag_cl, 'r--', label='Lazo Cerrado')
        ax1.set_ylabel('Magnitud (dB)'); ax1.grid(True, which="both", alpha=0.3); ax1.legend()

        # Fase
        ax2.semilogx(w, phase_ol, 'b', label='Lazo Abierto')
        ax2.semilogx(w_cl, phase_cl, 'r--', label='Lazo Cerrado')
        ax2.set_ylabel('Fase (grados)'); ax2.set_xlabel('Frecuencia (rad/s)'); ax2.grid(True, which="both", alpha=0.3)

        st.pyplot(fig)

    # 3. IMPULSO
    with tabs[2]:
        st.subheader("Respuesta al Impulso")
        _, y_imp_ol = signal.impulse(sys_ol, T=t)
        _, y_imp_cl = signal.impulse(sys_cl, T=t)

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(t, y_imp_ol, 'b', label='Lazo Abierto')
        ax.plot(t, y_imp_cl, 'r--', label='Lazo Cerrado')
        ax.set_title("Respuesta ante Delta de Dirac")
        ax.grid(True, alpha=0.3); ax.legend()
        st.pyplot(fig)

    # 4. ESCALÓN
    with tabs[3]:
        st.subheader("Respuesta al Escalón")
        _, y_step_ol = signal.step(sys_ol, T=t)
        _, y_step_cl = signal.step(sys_cl, T=t)

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(t, y_step_ol, 'b', label='Lazo Abierto')
        ax.plot(t, y_step_cl, 'r--', label='Lazo Cerrado')
        ax.set_title("Respuesta ante Escalón Unitario")
        ax.grid(True, alpha=0.3); ax.legend()
        st.pyplot(fig)

    # 5. RAMPA
    with tabs[4]:
        st.subheader("Respuesta a la Rampa")
        # Rampa unitaria: u(t) = t
        _, y_ramp_ol, _ = signal.lsim(sys_ol, t, t)
        _, y_ramp_cl, _ = signal.lsim(sys_cl, t, t)

        fig, ax = plt.subplots(figsize=(8, 4))
        ax.plot(t, t, 'k:', label='Entrada (Rampa)', alpha=0.5)
        ax.plot(t, y_ramp_ol, 'b', label='Lazo Abierto')
        ax.plot(t, y_ramp_cl, 'r--', label='Lazo Cerrado')
        ax.set_title("Respuesta ante Entrada Rampa")
        ax.grid(True, alpha=0.3); ax.legend()
        st.pyplot(fig)

"""
with open("app.py", "w") as f: f.write(codigo_app)

# 5. LANZAMIENTO
print("🚀 Iniciando Streamlit...")
subprocess.Popen(["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0", "--server.headless=true"])
time.sleep(3) # Espera de seguridad

if not os.path.exists("cloudflared-linux-amd64"):
    !wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
    !chmod +x cloudflared-linux-amd64

print("🔗 Creando enlace...")
with open("tunnel.log", "w") as log_file:
    subprocess.Popen(["./cloudflared-linux-amd64", "tunnel", "--url", "http://localhost:8501"], stdout=log_file, stderr=log_file)

found_url = None
for i in range(15):
    time.sleep(2)
    try:
        with open("tunnel.log", "r") as f:
            content = f.read()
            match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
            if match: found_url = match.group(0); break
    except: pass
    print(f"   ⏳ ({i+1})")

if found_url: print(f"\n✅ ENTRA AQUÍ: {found_url}")
else: print("❌ Error de conexión.")

🧹 Limpiando procesos...
🛠️ Instalando librerías...
🎵 Verificando audio...


✅ Audio descargado.
📝 Escribiendo aplicación completa...
🚀 Iniciando Streamlit...
🔗 Creando enlace...
   ⏳ (1)
   ⏳ (2)

✅ ENTRA AQUÍ: https://machine-ent-talk-golf.trycloudflare.com
